# What is Akka.Streams?
[Akka.Streams](https://getakka.net/articles/streams/introduction.html) is a high-level abstraction built on top of Akka.NET actors; it's general purpose implementation of the [Reactive Streams specification](https://www.reactive-streams.org/) for .NET and offers a [variety of built-in components](https://getakka.net/articles/streams/builtinstages.html), high performance, a succinct LINQ-like syntax, and a highly composable and [customizable development experience](https://getakka.net/articles/streams/custom-stream-processing.html).

## What is Akka.Streams Used For?
There are three compelling reasons to want to use Akka.Streams:

1. **Making asynchronous producer-consumer models safe and effective** - through the use of Reactive Streams, and more specifically through the use of "backoff" mechanisms that tell producers when they are overwhelming a slower-moving consumer with ouptut, we can prevent asynchronous systems from failing by helping producers slow down, aggregate, combine, debounce, or otherwise filter their output to help match the downstream input rates. This is done dynamically, so when consumers begin to move faster producers will also keep up.
2. **Providing re-usable asynchronous event-driven programming patterns** - Akka.Streams provides concise formulas for common event-driven patterns such as batching, debouncing, aggregating, filtering, and more.
3. **Shovel-ready integration with non-Akka.NET technologies** - through the [Alpakka project](https://alpakka.getakka.net/) and others it's easy to connect Akka.NET actors to Kafka, Azure EventHubs, AWS Kinesis / SNS / SQS, CSV files, SignalR, AMQP, and more.

This repository is mostly focused on the syntax for how to use Akka.Streams, how it works, and when you should use different portions of it.

## A Simple Example

To use Akka.Streams, we only need to install the [Akka.Streams NuGet package](https://www.nuget.org/packages/Akka.Streams/).

> PS> Install-Package Akka.Streams

And then we need to create an `ActorSystem`, since Akka.Streams uses Akka.NET actors internally.

In [ ]:
#r "nuget: Akka.Streams, 1.4.24"
using Akka.Actor;

var actorSystem = ActorSystem.Create("StreamsExample");

Installed Packages Akka.Streams, 1.4.24

Next, we're going to define [the basic components you need in order to run an Akka.NET Stream](https://getakka.net/articles/streams/basics.html#defining-and-running-streams):

1. A [`Source<T>` stage](https://getakka.net/api/Akka.Streams.Dsl.Source-2.html), which emits events - something as simple as an `IEnumerable<T>` can be converted into a `Source<T>`.
2. Optionally, one or more [`Flow<T>` stages](https://getakka.net/api/Akka.Streams.Dsl.Flow-3.html) which can be used to do all sorts of things: filtering, aggregating, projections, buffering, and so on.
3. Finally, a [`Sink<T>` stage](https://getakka.net/api/Akka.Streams.Dsl.Sink-2.html) - which is where the events in the stream complete processing. The simplest way to imagine a `Sink<T>` is as an asychronous `foreach` loop.

In this instance, we're going to create a really simple stream using a `List<int>` and some small processors.


In [ ]:
using Akka;
using Akka.Streams;
using Akka.Streams.Dsl;
using System.Collections.Immutable;

var ints = new List<int>(){ 1,2,3,4,5,6,7,8,9,19 };


// all streams start with one or more sources
Source<int, NotUsed> source = Source.From(ints);

// create a Flow that accepts an int and produces an int
// this Flow filters out any odd-numbered integers
Flow<int,int,NotUsed> flow = Flow.Create<int>().Where(x => x % 2 == 0);

// create a Sink that will write our integer output to the console
Sink<int, Task<IImmutableList<int>>> sink = Sink.Seq<int>();

We've just defined three separate Akka.Streams "stages" that we are going to connect together into a "graph".

```
Source<int> --> Flow<int> --> Sink<int, Task<IImmutableList<int>>>
```

As the code is defined right now, these stages can't actually do much - we have to connect them together and then use the "materializer," the Akka.Streams compiler, to create the stream and begin running it.

> **N.B.** `NotUsed` is our "materialization" value - all streams have the ability to potentailly be materialized into some other usable form, such as a `Task<T>` or an `IActorRef` - depending upon which stages you use and how you compile the stream together.

In this case, our stream is going to be materialized into a `Task` which will return an `IImmutableList<int>` that contains all of the values from the `Source<int>` that were not filtered out by the `Flow<int, int>` we connected to it.

In [ ]:
// create an instance of the materializer from the ActorSystem
// this will create the underlying actors as children of the /user root actor.
IMaterializer materializer = actorSystem.Materializer();

// you can also create materializers in the contexts of other actors, to create those
// stream actors as children of those contexts - this has some advantages from an
// encapsulation point of view and we'll learn how to do that later on.

// connect all of the stream stages together and return the materialized Task
// created by our Sink<int, Task<IImmutableList<int>>> stage
Task<IImmutableList<int>> allNumsTask = source.Via(flow).RunWith(sink, materializer);

IImmutableList<int> allNums = await allNumsTask;

foreach(var i in allNums){
    Console.WriteLine("{0}", i);
}

2
4
6
8


As you can see, only the even numbers are printed. 

This is a simple but complete Akka.Streams graph:

* All of the inputs and outputs are statically typed;
* An input can only connect to an output of the same type;
* Some stages, such as `Flow<TIn, TOut>` have both an input _and_ an output - those can be _different_ types; and
* In order to run the graph, it has to be "complete" - meaning that all inputs and outputs are connected.

When we make the call to `RunWith` that will cause the materializer to create the actors that are going to run the code defined in your stages. In this example, because the `Source<T>` is just an `IEnumerable<T>` this stream will be a finite stream - meaning it has a definite beginning and end. Once the `Source<int>` reaches the end, the stream will complete and terminate.

> Behind the scenes, when we called `RunWith` and passed in an instance of the `IMaterializer` - we actually created one or two actors who ran this workload and then self-terminated once they reached the end of the `IEnumerable<int>` that we used to create our `Source<int>`.